Augmentation 

In [2]:
import cv2
import os
from tqdm import tqdm
import numpy as np

Direcciones (rutas)

In [9]:
entrada = r'Datos\FiltrosA'
salida = r'Datos\Augmented'

Conjuntos y clases

In [16]:
conjuntos = ["conjunto_raw", "conjunto_bilateral", "conjunto_canny"]
clases = ["Covid", "Normal", "Neumonia"]

Función de augmentación

In [13]:
def aumentar_imagen(img):
    h, w = img.shape[:2]
    augmentadas = []

    # Rotaciones
    for angulo in [15, -15]:
        M = cv2.getRotationMatrix2D((w//2, h//2), angulo, 1)
        rotada = cv2.warpAffine(img, M, (w, h))
        augmentadas.append(rotada)

    # Zoom + recorte central
    zoom = cv2.resize(img, None, fx=1.2, fy=1.2)
    zh, zw = zoom.shape[:2]
    zoom_crop = zoom[(zh-h)//2:(zh-h-h)%2+h, (zw-w)//2:(zw-w-w)%2+w]
    augmentadas.append(zoom_crop)

    # Traslación
    M_trans = np.float32([[1, 0, 10], [0, 1, 10]])
    traslada = cv2.warpAffine(img, M_trans, (w, h))
    augmentadas.append(traslada)

    # Volteo horizontal
    flip = cv2.flip(img, 1)
    augmentadas.append(flip)

    return augmentadas

Contadores:

In [14]:
conteo_original = {}
conteo_final = {}

Proceso

In [17]:
for conjunto in conjuntos:
    conteo_original = {}
    conteo_final = {}

    for clase in clases:
        carpeta_entrada = os.path.join(entrada, conjunto, clase)
        carpeta_salida = os.path.join(salida, conjunto, clase)
        os.makedirs(carpeta_salida, exist_ok=True)

        imagenes = [f for f in os.listdir(carpeta_entrada) if f.endswith(('.png', '.jpg', '.jpeg'))]
        conteo_original[clase] = len(imagenes)

        for nombre_img in tqdm(imagenes, desc=f'Procesando {conjunto}/{clase}'):
            ruta_img = os.path.join(carpeta_entrada, nombre_img)
            img = cv2.imread(ruta_img)

            if img is None:
                print(f"Error leyendo imagen: {ruta_img}")
                continue

            base_name = os.path.splitext(nombre_img)[0]
            cv2.imwrite(os.path.join(carpeta_salida, f'{base_name}_orig.jpg'), img)

            augmentadas = aumentar_imagen(img)
            for i, aug_img in enumerate(augmentadas):
                cv2.imwrite(os.path.join(carpeta_salida, f'{base_name}_aug{i+1}.jpg'), aug_img)

        conteo_final[clase] = conteo_original[clase] * 6  # original + 5 augmentadas

    # Mostrar conteo por conjunto
    print(f"\nResumen del conjunto: {conjunto}")
    for clase in clases:
        print(f"  - {clase}: originales = {conteo_original[clase]}, aumentadas = {conteo_final[clase]}")

Procesando conjunto_raw/Neumonia: 100%|██████████| 70/70 [00:03<00:00, 20.07it/s]



Resumen del conjunto: conjunto_raw
  - Covid: originales = 111, aumentadas = 666
  - Normal: originales = 70, aumentadas = 420
  - Neumonia: originales = 70, aumentadas = 420


Procesando conjunto_bilateral/Neumonia: 100%|██████████| 70/70 [00:03<00:00, 21.84it/s]



Resumen del conjunto: conjunto_bilateral
  - Covid: originales = 111, aumentadas = 666
  - Normal: originales = 70, aumentadas = 420
  - Neumonia: originales = 70, aumentadas = 420


Procesando conjunto_canny/Neumonia: 100%|██████████| 70/70 [00:02<00:00, 24.47it/s]


Resumen del conjunto: conjunto_canny
  - Covid: originales = 111, aumentadas = 666
  - Normal: originales = 70, aumentadas = 420
  - Neumonia: originales = 70, aumentadas = 420
